### 한국, 미국 주식 휴장일 리스트 구하기

- 휴장일을 제외한 자료 사전 처리에 사용하기 위한 휴장일 수집(2020~2022)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/stock_market_holydays/sm_holydays.csv')

In [3]:
kor_days = df['KOR'][df['KOR'].notnull()].values.tolist()
usa_days = df['USA'][df['USA'].notnull()].values.tolist() # Nan value를 제외한 값을 리스트로 변환

In [9]:
def allsatsundays(start_date, end_date):
    day_sat = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SAT').strftime('%Y-%m-%d').tolist()
    day_sun = pd.date_range(start=start_date, end=end_date, 
                         freq='W-SUN').strftime('%Y-%m-%d').tolist()
    days = day_sat+day_sun
    days.sort()
    return days

In [5]:
start_date = '2020-01-01'
end_date = '2022-12-31'
holydays = allsatsundays('2020-01-01','2020-04-01')

In [6]:
kor_days = sorted(set(kor_days + holydays))
usa_days = sorted(set(usa_days + holydays))